In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

In [ ]:
model_fp32 = './vgg16-12.onnx'
model_quant = './vgg16-12.quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant)
